https://stepik.org/lesson/1576202/step/6

In [ ]:
# @title 🔧 Установка модуля и создание словаря для теста
!pip install navec
from navec import Navec
import requests

url_navec_hudlit = 'https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar'

response = requests.get(url_navec_hudlit, stream=True)
with open('navec_hudlit_v1_12B_500K_300d_100q.tar', 'wb') as f:
    f.write(response.content)

path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
global_navec = Navec.load(path)

In [ ]:
# Проверка
is_word = 'каждый' in global_navec  # проверка наличия слова в словаре
v = global_navec['каждый']          # embedding слова 'каждый'
indx = global_navec.vocab['каждый'] # индекс слова 'каждый' в словаре

is_word, v[:3], indx

(True, array([ 0.05372348, -0.0813841 , -0.51375115], dtype=float32), 161025)

In [ ]:
# Глобальная переменная для тестировввания
_global_var_text = [
"Как я отмечал во введении простейшая НС представляет собой",
"Это классический пример нейро сети",
"Каждая связь между нейронами имеет определенный"
]

In [ ]:
import torch
import torch.nn as nn
import torch.utils.data as data


class WordsDataset(data.Dataset):
    def __init__(self, prev_words=4):
        super().__init__()
        self.text = _global_var_text
        df = []
        for string in self.text:
            s = [word for word in string.lower().split()]
            for i in range(prev_words, len(s)):
                df.append(s[i-prev_words:i+1])
        self.df = df
        self.length = len(self.df)

    def __len__(self):
        return self.length

    def __getitem__(self, indx):
            prev_words = self.df[indx][:-1]
            prev_words_emb = torch.stack([torch.as_tensor(global_navec[word]) for word in prev_words])
            next_word = self.df[indx][-1]
            next_word_indx = global_navec.vocab[next_word]

            return prev_words_emb, torch.tensor(next_word_indx)

d_train = WordsDataset(prev_words=4)

train_data = data.DataLoader(d_train, batch_size=8, shuffle=True)

# Проверка
# Проверка
print(d_train[3])
print(d_train[4])
print(len(d_train))
# [['как', 'я', 'отмечал', 'во', 'введении'],
#  ['я', 'отмечал', 'во', 'введении', 'простейшая'],
#  ['отмечал', 'во', 'введении', 'простейшая', 'нс'],
#  ['во', 'введении', 'простейшая', 'нс', 'представляет'],
#  ['введении', 'простейшая', 'нс', 'представляет', 'собой'],
#  ['это', 'классический', 'пример', 'нейро', 'сети'],
#  ['каждая', 'связь', 'между', 'нейронами', 'имеет'],
#  ['связь', 'между', 'нейронами', 'имеет', 'определенный']]

(tensor([[-0.0204, -0.4305, -0.3130,  ..., -0.5250,  0.1350,  0.2226],
        [-0.4001,  0.3622, -0.3973,  ..., -0.0571,  0.1050,  0.3030],
        [-0.0319,  0.4600,  0.1253,  ...,  0.5209,  0.2278, -0.2517],
        [ 0.5255,  0.2983,  0.3089,  ..., -0.2217,  0.1690, -0.0966]]), tensor(332056))
(tensor([[-0.4001,  0.3622, -0.3973,  ..., -0.0571,  0.1050,  0.3030],
        [-0.0319,  0.4600,  0.1253,  ...,  0.5209,  0.2278, -0.2517],
        [ 0.5255,  0.2983,  0.3089,  ..., -0.2217,  0.1690, -0.0966],
        [-0.0460,  0.0078,  0.3391,  ...,  0.0788,  0.3901,  0.4029]]), tensor(407059))
8


In [ ]:
# @title Эталонное решение для проверки
import torch
import torch.nn as nn
import torch.utils.data as data

class WordsDataset(data.Dataset):
    def __init__(self, navec_emb, prev_words=4):
        self.prev_words = prev_words
        self.navec_emb = navec_emb

        self.lines = _global_var_text
        self.vocab = set((" ".join(self.lines)).lower().split())
        self.vocab_size = len(self.vocab)

        data = []
        targets = []

        for t in self.lines:
            words = t.lower().split()
            for item in range(len(words)-self.prev_words):
                data.append([self.navec_emb[words[x]].tolist() for x in range(item, item + self.prev_words)])
                targets.append(self.navec_emb.vocab[words[item+self.prev_words]])

        self.data = torch.tensor(data)
        self.targets = torch.tensor(targets)

        self.length = len(data)

    def __getitem__(self, item):
        return self.data[item], self.targets[item]

    def __len__(self):
        return self.length


d_train = WordsDataset(global_navec)
train_data = data.DataLoader(d_train, batch_size=8, shuffle=True)

# Проверка
print(d_train[3])
print(d_train[4])
print(len(d_train))

(tensor([[-0.0204, -0.4305, -0.3130,  ..., -0.5250,  0.1350,  0.2226],
        [-0.4001,  0.3622, -0.3973,  ..., -0.0571,  0.1050,  0.3030],
        [-0.0319,  0.4600,  0.1253,  ...,  0.5209,  0.2278, -0.2517],
        [ 0.5255,  0.2983,  0.3089,  ..., -0.2217,  0.1690, -0.0966]]), tensor(332056))
(tensor([[-0.4001,  0.3622, -0.3973,  ..., -0.0571,  0.1050,  0.3030],
        [-0.0319,  0.4600,  0.1253,  ...,  0.5209,  0.2278, -0.2517],
        [ 0.5255,  0.2983,  0.3089,  ..., -0.2217,  0.1690, -0.0966],
        [-0.0460,  0.0078,  0.3391,  ...,  0.0788,  0.3901,  0.4029]]), tensor(407059))
8


In [ ]:
# https://stepik.org/lesson/1576202/step/6?discussion=11320238&thread=solutions&unit=1597477
import re
import torch
import torch.nn as nn
import torch.utils.data as data

class WordsDataset(data.Dataset):
    def __init__(self, navec_emb, prev_words):
        self.prev_words = prev_words
        self.navec_emb = navec_emb
        self.text = _global_var_text
        self.text = " ".join(self.text)
        self.text = self.text.replace('\n', ' ')
        self.text = re.sub(r'[^А-яA-z- ]', '', self.text)
        self.words = self.text.lower().split()
        self.words = [word for word in self.words if word in self.navec_emb]
        vocab = set(self.words)

        self.int_to_word = dict(enumerate((vocab)))
        self.word_to_int = {b: a for a, b in self.int_to_word.items()}
        self.vocab_size = len(vocab)


    def __getitem__(self, item):
        _data = torch.stack([torch.tensor(self.navec_emb[self.words[x]])
                              for x in range(item, item+self.prev_words)])
        word = self.words[item + self.prev_words]
        index = self.navec_emb.vocab[word]

        return _data, torch.tensor(index)


    def __len__(self):
        return len(self.words) - self.prev_words



d_train = WordsDataset(global_navec, prev_words=4)
train_data = data.DataLoader(d_train, batch_size=8, shuffle=True)

# Проверка
print(d_train[3])
print(d_train[4])
print(len(d_train))

(tensor([[-0.0204, -0.4305, -0.3130,  ..., -0.5250,  0.1350,  0.2226],
        [-0.4001,  0.3622, -0.3973,  ..., -0.0571,  0.1050,  0.3030],
        [-0.0319,  0.4600,  0.1253,  ...,  0.5209,  0.2278, -0.2517],
        [ 0.5255,  0.2983,  0.3089,  ..., -0.2217,  0.1690, -0.0966]]), tensor(332056))
(tensor([[-0.4001,  0.3622, -0.3973,  ..., -0.0571,  0.1050,  0.3030],
        [-0.0319,  0.4600,  0.1253,  ...,  0.5209,  0.2278, -0.2517],
        [ 0.5255,  0.2983,  0.3089,  ..., -0.2217,  0.1690, -0.0966],
        [-0.0460,  0.0078,  0.3391,  ...,  0.0788,  0.3901,  0.4029]]), tensor(407059))
16
